# CS584 NLP HW3; Task: Machine Translation
Milan Chandiramani CWID: 20032010

Goals:
The goal of HW3 is for you to get hands-on experience of utilizing Seq2Seq model for machine translation task. You will get a deeper understanding of how the input sequence is translated to the output sequence with the Seq2Seq model. The skills of you learnt in this homework will benefit your understanding of a wide range of NLP tasks beyond machine translation. Please feel
free to use any packages or libraries in your implementation.

Similar to HW1 and HW2, all questions are open questions and there is no fixed solution. The difference in data selection and processing, parameter initialization, data split, etc., will lead to the differences in predictions and evaluation results. Therefore, during the grading, the specific
values in the results are not required. It is important that you focus on implementing and setting up the pipelines of applying these models to solve the tasks.

In [ ]:
# Install datasets library
!pip install datasets==3.6.0
!pip install transformers
!pip install torchtext
!pip install torch
!pip install tensorflow
!pip install sentencepiece
!pip install sacremoses
!pip install sacrebleu
!pip install ctranslate2

In [4]:
import pandas as pd
from datasets import load_dataset
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate import bleu_score

## 1. Data preparation (5 points)
### 1) Data download and preprocessing:
Download the Flores200 dataset, which is a benchmark data for machine translation between English and low-resource languages.
More details about the data are available here. Please select English and another language (based on your knowledge or background) as the target language. Further, please select 100 sentence pairs (based on the source and target languages) for this homework.
You can directly download the data with the following codes from Hugging Face. You will need to install 🤗 Transformers and 🤗 Datasets first. A quick tour to set up and run Transformers on Hugging Face can be found here.

In [5]:
# # Download the Flores200 dataset
from datasets import load_dataset
dataset = load_dataset("Muennighoff/flores200", 'all')

#Above code no longer works and gives error for above I believe. Just used English to Hindi
# dataset = load_dataset("Muennighoff/flores200", "eng_Latn-hin_Deva")
# dataset = load_dataset("facebook/flores", "eng_Latn")
# print("Dataset loaded successfully.")

The repository for Muennighoff/flores200 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Muennighoff/flores200.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating dev split: 0 examples [00:00, ? examples/s]

Generating devtest split: 0 examples [00:00, ? examples/s]

In [6]:
# Select English as source language and Hindi as target language for translation
source_language = dataset['dev']['sentence_eng_Latn']
target_language = dataset['dev']['sentence_hin_Deva']

In [7]:
# Combine source and target sentences into pairs
sentence_pairs = list(zip(source_language, target_language))

# Randomly select 100 sentence pairs
random.seed(50)
selected_pairs = random.sample(sentence_pairs, 100)

In [8]:
# Define the list for source and target sentences
source_sentences = []
target_sentences = []

# Display the selected sentence pairs
for idx, (source, target) in enumerate(selected_pairs):
    source_sentences.append(source)
    target_sentences.append(target)
    print(f"Pair {idx + 1} - Source: {source}\n         Target: {target}\n")

Pair 1 - Source: It is not a learning disability, it is a learning disorder; it "affects 3 to 5 percent of all children, perhaps as many as 2 million American children".
         Target: यह सीखने की अक्षमता नहीं है, यह सीखने का विकार है; यह “3 से 5 प्रतिशत बच्चों को प्रभावित करता है, संभवतः लगभग 2 मिलियन अमेरिकी बच्चों को”.

Pair 2 - Source: For those unfamiliar with medical jargon, the words infectious and contagious have distinct meanings.
         Target: जो मेडिकल शब्दजाल से अपरिचित हैं, उन लोगों के लिए संक्रामक और सांसर्गिक शब्दों के अलग-अलग अर्थ हैं.

Pair 3 - Source: Some studies suggest that the disease must become less lethal before it can cause a global epidemic, he noted.
         Target: उन्होंने कहा कि कुछ अध्ययनों से पता चलता है कि वैश्विक महामारी का कारण बनने से पहले बीमारी कम घातक हो सकती है.

Pair 4 - Source: Other scientists argue that these plants are very poisonous so it is unlikely that any dinosaur ate them, even though today the sloth and other animals like the p

## 2. Machine Translation with Seq2Seq model (65 points, 20 points for each model, 5 points for data statistics)
Please use the following models to perform machine translation on the data you prepared.
• Feel free to directly utilize the existing implementations on Hugging Face.

• There is no specific requirement for the parameter settings. You are encouraged to try and test different settings and report the results. For other settings that are not specified here, you have the flexibility to select.

• If you want to perform the translation more efficiently with the following models, you can consider using CTranslate2. Its Github repository is available here. The documentation of the package is available here. M2M-100 and MBART-50 are under Fairseq.

• During the implementation, for each model, you will need to specify the source language and target language or select the specific model for your source and target languages

### 1) OPUS-MT: Go to the page of Hugging Face, click ‘Models’ on the top of the page. Search
for the OPUS-MT model based on the languages you selected. For example, if your
source language is English, and target language is French, you can search for ‘opus-mt-en-fr’, which will be the model you can use. The list of supported language pairs can be found here.
a. Once you identified your model, click the model and go to the corresponding page, you will see a button about ‘Use in Transformers’ (top right of the page), which shows examples of how to use the model. See the attached two screenshots.
b. Implementation details about the tokenizer and model can be found here.

In [9]:
# Load a pre-trained model and tokenizer for translation from English to German
opus_tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-hi')
opus_model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
# Function for translation
def opus_translate_sentence(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model.generate(**inputs)
    translated_sentence = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return translated_sentence

# Define the list to store the translated sentences
opus_translated_sentences_list = []

# Perform translations on the 100 sentence pairs
for idx, (source_language, target_language) in enumerate(selected_pairs):
    opus_translated_sentence = opus_translate_sentence(source_language, opus_model, opus_tokenizer)
    opus_translated_sentences_list.append(opus_translated_sentence)

    print(f"Pair {idx + 1}")
    print(f"Source: {source_language}")
    print(f"Target: {target_language}")
    print(f"Translated: {opus_translated_sentence}\n")

Pair 1
Source: It is not a learning disability, it is a learning disorder; it "affects 3 to 5 percent of all children, perhaps as many as 2 million American children".
Target: यह सीखने की अक्षमता नहीं है, यह सीखने का विकार है; यह “3 से 5 प्रतिशत बच्चों को प्रभावित करता है, संभवतः लगभग 2 मिलियन अमेरिकी बच्चों को”.
Translated: यह एक सीखने की असमर्थता नहीं है, यह एक सीखने की विकार है; यह "सभी बच्चों में से ३ से ५ प्रतिशत बच्चों में से एक है, शायद दस लाख से अधिक अमरीकी बच्चे.

Pair 2
Source: For those unfamiliar with medical jargon, the words infectious and contagious have distinct meanings.
Target: जो मेडिकल शब्दजाल से अपरिचित हैं, उन लोगों के लिए संक्रामक और सांसर्गिक शब्दों के अलग-अलग अर्थ हैं.
Translated: जिन लोगों को चिकित्सा - पत्र की कमी है, उनके लिए शब्द संक्रामक और संक्रामक हैं ।

Pair 3
Source: Some studies suggest that the disease must become less lethal before it can cause a global epidemic, he noted.
Target: उन्होंने कहा कि कुछ अध्ययनों से पता चलता है कि वैश्विक महामारी का कार

### 2) M2M-100: M2M100 is a multilingual encoder-decoder (seq-to-seq) model trained for
Many-to-Many multilingual translation. It was introduced in the paper Beyond English-Centric Multilingual Machine Translation (https://arxiv.org/abs/2010.11125) and first released in the repository: (https://github.com/facebookresearch/fairseq/tree/main/examples/m2m_100).

a. Implementation details about the tokenizer and model can be found at: https://huggingface.co/facebook/m2m100_418M

In [11]:
# load a pre-trained model and tokenizer for the M2M100 model
m2m_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
m2m_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [13]:
# translate English to Hindi
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m_tokenizer.src_lang = "en"
# Function for translation
def m2m_translate_sentence(sentence, model, tokenizer):
    input_tokens = tokenizer(sentence, return_tensors="pt")
    output_tokens = model.generate(**input_tokens, forced_bos_token_id=tokenizer.get_lang_id("hi"))
    translated_sentence = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
    return translated_sentence

# Define the list to store the translated sentences
m2m_translated_sentences_list = []

# Perform translations on the 100 sentence pairs
for idx, (source_language, target_language) in enumerate(selected_pairs):
    m2m_translated_sentence = m2m_translate_sentence(source_language, m2m_model, m2m_tokenizer)
    m2m_translated_sentences_list.append(m2m_translated_sentence)

    print(f"Pair {idx + 1}")
    print(f"Source: {source_language}")
    print(f"Target: {target_language}")
    print(f"Translated: {m2m_translated_sentence}\n")

Pair 1
Source: It is not a learning disability, it is a learning disorder; it "affects 3 to 5 percent of all children, perhaps as many as 2 million American children".
Target: यह सीखने की अक्षमता नहीं है, यह सीखने का विकार है; यह “3 से 5 प्रतिशत बच्चों को प्रभावित करता है, संभवतः लगभग 2 मिलियन अमेरिकी बच्चों को”.
Translated: यह एक सीखने की विकार नहीं है, यह एक सीखने की विकार है; यह "सभी बच्चों के 3 से 5 प्रतिशत को प्रभावित करता है, विशेष रूप से 2 मिलियन अमेरिकी बच्चों के रूप में।

Pair 2
Source: For those unfamiliar with medical jargon, the words infectious and contagious have distinct meanings.
Target: जो मेडिकल शब्दजाल से अपरिचित हैं, उन लोगों के लिए संक्रामक और सांसर्गिक शब्दों के अलग-अलग अर्थ हैं.
Translated: चिकित्सा जार्गन के साथ अज्ञात लोगों के लिए, संक्रामक और संक्रामक शब्दों के अलग-अलग अर्थ हैं।

Pair 3
Source: Some studies suggest that the disease must become less lethal before it can cause a global epidemic, he noted.
Target: उन्होंने कहा कि कुछ अध्ययनों से पता चलता है कि वै

### 3) MBART-50:
mbart-large-50-many-to-many-mmt is fine-tuned based on mBART-large-50 (https://huggingface.co/facebook/mbart-large-50) for multilingual machine translation. It was introduced in the paper about Multilingual Translation with Extensible Multilingual Pretraining and Finetuning (https://arxiv.org/abs/2008.00401).
a. Implementation details about the tokenizer and model can be found at : https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt

In [14]:
# load a pre-trained model and tokenizer for the MBART-50 model
mbart_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
mbart_tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [18]:
# translation (English to Hindi)
def mbart_translate_sentence(sentence, model, tokenizer):
    tokenizer.src_lang = "en_XX"
    input_tokens = tokenizer(sentence, return_tensors="pt")
    output_tokens = model.generate(**input_tokens, forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
    translated_sentence = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
    return translated_sentence

# Define the list to store the length of the translated sentences
mbart_translated_sentences_list = []

# Perform translations on the 100 sentence pairs
for idx, (source_language, target_language) in enumerate(selected_pairs):
    mbart_translated_sentence = mbart_translate_sentence(source_language, mbart_model, mbart_tokenizer)
    mbart_translated_sentences_list.append(mbart_translated_sentence)

    print(f"Pair {idx + 1}")
    print(f"Source: {source_language}")
    print(f"Target: {target_language}")
    print(f"Translated: {mbart_translated_sentence}\n")


Pair 1
Source: It is not a learning disability, it is a learning disorder; it "affects 3 to 5 percent of all children, perhaps as many as 2 million American children".
Target: यह सीखने की अक्षमता नहीं है, यह सीखने का विकार है; यह “3 से 5 प्रतिशत बच्चों को प्रभावित करता है, संभवतः लगभग 2 मिलियन अमेरिकी बच्चों को”.
Translated: यह एक सीखने में असमर्थता नहीं है, यह एक सीखने में विकार है; यह सभी बच्चों में 3 से 5 प्रतिशत को प्रभावित करता है, शायद 2 मिलियन अमेरिकी बच्चों तक।

Pair 2
Source: For those unfamiliar with medical jargon, the words infectious and contagious have distinct meanings.
Target: जो मेडिकल शब्दजाल से अपरिचित हैं, उन लोगों के लिए संक्रामक और सांसर्गिक शब्दों के अलग-अलग अर्थ हैं.
Translated: उन लोगों के लिए, जो चिकित्सा शब्दावली से परिचित नहीं हैं, संक्रामक और संक्रामक शब्दों के अलग-अलग अर्थ हैं।

Pair 3
Source: Some studies suggest that the disease must become less lethal before it can cause a global epidemic, he noted.
Target: उन्होंने कहा कि कुछ अध्ययनों से पता चलता है कि

### 4) Data statistics:
Based on one of the tokenizers you used in the above three models, please provide some basic data statistics of the 100 sentences you selected. Such as the minimum/average/maximum length of the source sentences and target sentences. Feel
free to provide any other data statistics that are reasonable

In [19]:
def max_length(sentences_list):

    max_length = len(sentences_list[0])

    for sentence in sentences_list:
        if max_length < len(sentence):
            max_length = len(sentence)

    return max_length

def min_length(sentences_list):

    min_length = len(sentences_list[0])

    for sentence in sentences_list:
        if min_length > len(sentence):
            min_length = len(sentence)

    return min_length

def avg_length(sentences_list):

    sum = 0

    for sentence in sentences_list:
        sum += len(sentence)

    avg_length = sum // len(sentences_list)

    return avg_length

In [20]:
# Define the columns and rows of the table containing the statistics of the source and target sentences
source_target_sentences_length_table = {
    'Sentences' : ['Source sentences',
                   'Target sentences'],
    'Maximum length of the sentences' : [max_length(source_sentences),
                                         max_length(target_sentences)],
    'Minimum length of the sentences' : [min_length(source_sentences),
                                         min_length(target_sentences)],
    'Average length of the sentences' : [avg_length(source_sentences),
                                         avg_length(target_sentences)],
}

# Create the dataframe of the statistics of the source and target sentences
source_target_sentences_statistics_table = pd.DataFrame(source_target_sentences_length_table)

In [21]:
# Define the columns and rows of the table containing the statistics of the translated sentences
translated_sentences_length_table = {
    'Models' : ['OPUS_MT',
                'M2M-100',
                'MBART-50'],
    'Minimum length of the translated sentences' : [min_length(opus_translated_sentences_list),
                                                    min_length(m2m_translated_sentences_list),
                                                    min_length(mbart_translated_sentences_list)],
    'Average length of the translated sentences' : [avg_length(opus_translated_sentences_list),
                                                    avg_length(m2m_translated_sentences_list),
                                                    avg_length(mbart_translated_sentences_list)],
    'Maximum length of the translated sentences' : [max_length(opus_translated_sentences_list),
                                                    max_length(m2m_translated_sentences_list),
                                                    max_length(mbart_translated_sentences_list)],
}

# Create the dataframe of the statistics of the translated sentences
translated_sentences_statistics_table = pd.DataFrame(translated_sentences_length_table)

In [22]:
#Displays the table containing the statistics of the source and target sentences
source_target_sentences_statistics_table

,Sentences,Maximum length of the sentences,Minimum length of the sentences,Average length of the sentences
0,Source sentences,273,28,132
1,Target sentences,338,36,130


In [23]:
#Displays the table containing the statistics of the translated sentences
translated_sentences_statistics_table

,Models,Minimum length of the translated sentences,Average length of the translated sentences,Maximum length of the translated sentences
0,OPUS_MT,18,112,215
1,M2M-100,36,133,311
2,MBART-50,26,131,336


## 3. Results analysis and evaluation (30 points, 10 points for each sub-question)

### 3.1) Please use the BLUE score to evaluate the performance of the models. You can find the details and examples about how to get BLUE scores in this link.

In [25]:
nltk.download('punkt_tab')
# Tokenize the target sentence
target_ref = [[word_tokenize(sentence) for sentence in target_sentences] for reference in target_sentences]

## Evaluation :
# Evaluation for OPUS-MT model
opus_predictions = [word_tokenize(sentence) for sentence in opus_translated_sentences_list]
opus_bleu = bleu_score.corpus_bleu(target_ref, opus_predictions)
print("The BLEU score of OPUS-MT model:")
print(opus_bleu)

# Perform evaluation for M2M-100 model
m2m_predictions = [word_tokenize(sentence) for sentence in m2m_translated_sentences_list]
m2m_bleu = bleu_score.corpus_bleu(target_ref, m2m_predictions)
print("\nThe BLEU score of M2M-100 model:")
print(m2m_bleu)

# Perform evaluation for MBART-50 model
mbart_predictions = [word_tokenize(sentence) for sentence in mbart_translated_sentences_list]
mbart_bleu = bleu_score.corpus_bleu(target_ref, mbart_predictions)
print("\nThe BLEU score of MBART-50 model:")
print(mbart_bleu)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


The BLEU score of OPUS-MT model:
0.211393946601422

The BLEU score of M2M-100 model:
0.31654173035805444

The BLEU score of MBART-50 model:
0.2993131727782203


### 3.2) Provide the comparison discussions and analysis based on the evaluation results you obtained from each model. For example, which model performs best, which performs worst, and what is the possible reason for such results.

Model Comparison and Analysis Based on BLEU Scores


BLEU Score for the models is as follows:
OPUS-MT	0.2114
M2M-100	0.3165
MBART-50	0.2993

A. OPUS-MT (BLEU = 0.2114)

OPUS-MT models are light-weight and efficient, but they underperformed as seen above in comparison with larger models like M2M or MBART and achieved the lowest BLEU score among the three systems, with a value of approximately 0.21 for English -> Hindi translation. This result indicates that OPUS-MT struggled to produce translations closely aligned with the reference sentences. Although OPUS uses OPUS corpora (Tatoeba, GlobalVoices, etc.), the scale is much smaller than Meta's large datasets. OPUS provides separate models per language pair, which can be good but can lead to inconsistent performance across domains, sometimes even overfitting to narrow datasets and no denoising pretraining or large-scale generative training like MBART and had difficulty capturing the complex morphology, word order variations, and context required English -> Hindi translation..



B. M2M-100 (BLEU = 0.3165)

M2M-100 model achieved the highest BLEU score, around 0.32, demonstrating the strongest translation performance for English -> Hindi among the models evaluated. This superior result reflects the model's training on a massive dataset covering 100+ languages with direct parallel sentence pairs, allowing it to learn fine-grained linguistic patterns for many language pairs, including English→Hindi. Unlike many multilingual models that route translations through English, M2M-100 is explicitly designed to translate directly between languages, which can significantly improve accuracy. This allows better handling of linguistic variation, morphology, and rare word patterns.


C. MBART-50 (BLEU = 0.2993)

The MBART-50 model produced a BLEU score of about 0.30, placing it in between the three models. This performance highlights MBART-50's strengths as a multilingual sequence-to-sequence model pretrained across 50 languages using a large denoising objective is able to generate fluent and contextually coherent translations for English -> Hindi, likely benefiting from its cross-lingual pretraining and its ability to capture long-range semantic relationships. However, its performance remains slightly below M2M-100, which may be because M2M-100 covers 100 languages, while mBART-50 covers 50. The larger pool gives M2M-100 more data and more opportunities for transfer learning, which likely gives it a slight edge. MBART often handles longer sentences better when compared to OPUS-MT which may perform poorly on complex syntax. These results are consistent with known benchmarks and seem logical as larger and more comprehensively trained multilingual transformers generally outperform lighter, language-pair–specific models.

Additional Possible Reasons for Comparatively Lower Score Differences
Language Dialect Difference (Dataset domain mismatch): If test dataset differs from the training domain (e.g., news vs. conversations or the variety of dialects (in this case, I saw persian inspired hindi words in the target sentences more often than sanskrit origin hindi words)
Also, 100 sentences is a very small sample size.

### 3.3) Select two data samples and compare the translation obtained by the three models with the ground truth. Provide some discussions based on your findings on these two examples.

In [28]:
# Retrieve 2 data samples
source_two_samples = source_sentences[27:29]
target_two_samples = target_sentences[27:29]
opus_two_samples = opus_translated_sentences_list[27:29]
m2m_two_samples = m2m_translated_sentences_list[27:29]
mbart_two_samples = mbart_translated_sentences_list[27:29]

# Display the sampled sentences
print("The first sample")
print("Source:", source_two_samples[0])
print("Target:", target_two_samples[0])
print("OPUS-MT translation:", opus_two_samples[0])
print("M2M-100 translation:", m2m_two_samples[0])
print("MBART-50 translation:", mbart_two_samples[0])
print("\nThe second sample")
print("Source:", source_two_samples[1])
print("Target:", target_two_samples[1])
print("OPUS-MT translation:", opus_two_samples[1])
print("M2M-100 translation:", m2m_two_samples[1])
print("MBART-50 translation:", mbart_two_samples[1])

The first sample
Source: The band's management company, HK Management Inc., gave no initial reason when they canceled on September 20, but blamed logistical reasons by the next day.
Target: बैंड की प्रबंधन कंपनी, एचके मैनेजमेंट इनकॉरपोरेशन, ने 20 सितंबर को अपने कार्यक्र्म को रद्द करने का कोई प्रारंभिक कारण नहीं बताया, लेकिन अगले दिन तैयारी के कारणों को दोषी ठहराया।
OPUS-MT translation: समूह के प्रबंधन कंपनी, HK प्रबंधन प्रबंधन., जब वे सितंबर 20 पर रद्द कर दिया लेकिन अगले दिन के लिए लॉग-पत्रों पर दोष लगाया.
M2M-100 translation: बैंड की प्रबंधन कंपनी, HK Management Inc. ने 20 सितंबर को रद्द होने पर कोई प्रारंभिक कारण नहीं दिया, लेकिन अगले दिन तक लॉगिस्टिक कारणों को दोषी ठहराया।
MBART-50 translation: बैंड के प्रबंधन कंपनी HK Management Inc. ने 20 सितंबर को रद्द करने का कोई प्रारंभिक कारण नहीं दिया, लेकिन अगले दिन logistical कारणों को दोषी ठहराया.

The second sample
Source: There is a surcharge for having more than 2 passengers, so this option might be more expensive than necessary.
Target

Analysis and Discussion:

**Sample 1**

1. OPUS-MT shows noticeable grammatical and lexical issues in this example. It mistranslates some of the key phrases. For instance, "gave no initial reason" doesn't exist in translation and breaks the grammatical flow. On the other hand "logistical reasons" is rendered as "लॉग-पत्रों" (literally means : log-letters) which is semantically wrong. The output also suffers from repetition ("प्रबंधन प्रबंधन") and unnatural word ordering. These errors indicate that OPUS-MT struggles with complex sentence structure, long-distance dependencies, and domain-specific terminology, matching its relatively low BLEU score.

2. M2M-100 produces a mostly correct and fluent translation. While it doesn't translate the company name at all, it correctly translates the sentence structure, handles the temporal clause ("by the next day"), and tries to render "logistical reasons" very close to actual modern hindi word transliteration of English words ("लॉजिस्टिक" is right and model gives : "लॉगिस्टिक"). The phrasing is natural, and the meaning aligns closely with the ground truth. The grammar is perfect, correctly identifying कंपनी (company) as feminine and using the correct possessive marker (बैंड **की** प्रबंधन कंपनी). This aligns with M2M-100's strong BLEU score, showing its ability to capture complex relationships in longer sentences.

3. MBART-50 also performs well but includes one small issue: it leaves the English word like the company name and "logistical" untranslated. While common in spoken Hinglish, it is a less professional translation than M2M-100's full transliteration. It also makes a small but clear grammatical error, using the masculine possessive marker (बैंड **के** प्रबंधन कंपनी). This is incorrect, as कंपनी is feminine. Other parts of the sentence are translated accurately and fluently, reflecting good semantic understanding but occasional vocabulary gaps. This aligns with MBART's tendency to do well on structure and fluency but occasionally miss specific lexical items. This single example perfectly captures why M2M-100 scored higher. mBART-50 got the gist right, but M2M-100 mastered the grammatical and lexical details.



**Sample 2**


1. OPUS-MT again produces a translation that is semantically a little off: it translates the surcharge as "एक भारी कीमत" (a heavy price), which exaggerates meaning (it sounds more like a dramatic movie dialog rather than normal speech) and shifts nuance away from a simple additional fee. It also introduces unnatural phrasing ("2 यात्रियों से भी ज़्यादा यात्री होने की"). While understandable, the translation is less precise and deviates from the target meaning.

2. M2M-100

M2M-100 produces a fluent and accurate translation, correctly using "अतिरिक्त शुल्क" to represent "surcharge." The latter half of the sentence ("आवश्यक से अधिक महंगा हो सकता है") closely mirrors the target's meaning. The translation is natural, concise, and faithful.

3. MBART-50

MBART-50 is nearly identical in quality to M2M-100 in this example. It correctly produces "अतिरिक्त प्रभार" (another valid translation of surcharge) and maintains smooth sentence flow. The translation is semantically accurate and stylistically natural. Both models perform comparably here.



Both, M2M-100 and MBART-50 models perfectly understood and translated the key term "surcharge."

M2M-100 used अतिरिक्त शुल्क (atirikt śulk - additional fee).

MBART-50 used अतिरिक्त प्रभार (atirikt prabhār - additional charge).

Both are 100% correct, formal Hindi translations for "surcharge." Interestingly, the human target used the English loanword सरचार्ज (sarchārj), but the models' translations are arguably "better" or at least more formal.
Also, another insightful observation we see is that the human target language use the word "ज़रूरत" to express "necessary" which is part of the persian inspired Hindi dialect (analogous to Urdu).
Whereas, both the models M2M-100 and MBART-50 used "आवश्यक" to express "necessary" which is a sansrit origin Hindi word.
This explains the drop in BLUE score lower than 0.32 .

**Overall Findings from Qualitative Analysis**

The qualitative inspection of both samples aligns well with the BLEU score rankings:

M2M-100 consistently produces accurate, fluent, and contextually aligned translations, especially for complex sentences and domain-specific terms.

MBART-50 also performs strongly, producing natural translations but occasionally leaving untranslated English words or making small lexical mistakes.

OPUS-MT is the least consistent, showing issues with grammar, word choice, and sentence structure, especially when the source sentences are longer or contain technical terms.

These example-based observations reinforce the conclusion that models trained on larger multilingual datasets with dense cross-lingual representations (like M2M-100 and MBART-50) outperform lighter, language-pair-specific systems like OPUS-MT, both quantitatively and qualitatively.

### 4. Self-Reflection (5 points)
This question is required in the submission to help you assess your understanding of the related topics and identify the areas for further improvement and set goals accordingly.

It also promotes communication between you and the instructor and will further improve the course development and learning experience.
Question Answers
(provide the answer index)

Q1: Rate your performance


4-Somewhat satisfied

Q2: How much does your understanding of the concepts influence
your performance?


5-Very important

Q3: How much does the design of the examples/questions (is relatable
to you) influence your performance?

4-Somewhat important


Q4: When you answer Q3, which are the primary factors influencing
the relatability of the examples to you?

3-Age

6- Hobbies and interests

7- Prior knowledge about the context in the example



REFERENCE:

[1] Muennighoff, N. (n.d.). flores200 dataset. Hugging Face. Retrieved from https://huggingface.co/datasets/Muennighoff/flores200

[2] BLEU metric for machine translation evaluation. Hugging Face. Retrieved from https://huggingface.co/spaces/evaluate-metric/bleu

[3] Facebook AI. (n.d.). mbart-large-50 model. Hugging Face. Retrieved from https://huggingface.co/facebook/mbart-large-50

[4] MarianMTModel documentation. Hugging Face. Retrieved from https://huggingface.co/docs/transformers/main/en/model_doc/marian#transformers.MarianMTModel

[5] Facebook/M2M100_418M. Retrieved from https://huggingface.co/facebook/m2m100_418M

[6] Facebook/MBART-large-50-many-to-many-mmt. Retrieved from https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt


In [ ]:
#--------------------------------------------- Thank you! EOF ---------------------------------------------#